In [1]:
import os
import pandas as pd
import numpy as np
DATA_PATH = "data/2019-2023"

In [2]:
year = 2023
pkl_name = os.path.join(DATA_PATH, str(year) + "_all.pkl")
df_2023 = pd.read_pickle(pkl_name)
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2493 entries, 0 to 2492
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2493 non-null   int64         
 1   date      2493 non-null   datetime64[ns]
 2   title     2493 non-null   object        
 3   authors   2493 non-null   object        
 4   keywords  2493 non-null   object        
 5   abstract  2289 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 117.0+ KB


In [3]:
year = 2022
pkl_name = os.path.join(DATA_PATH, str(year) + "_all.pkl")
df_2022 = pd.read_pickle(pkl_name)
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20851 entries, 0 to 20850
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        20851 non-null  int64         
 1   date      20851 non-null  datetime64[ns]
 2   title     20851 non-null  object        
 3   authors   20851 non-null  object        
 4   keywords  20851 non-null  object        
 5   abstract  19201 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 977.5+ KB


In [4]:
year = 2021
pkl_name = os.path.join(DATA_PATH, str(year) + "_all.pkl")
df_2021 = pd.read_pickle(pkl_name)
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22268 entries, 0 to 22267
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        22268 non-null  int64         
 1   date      22268 non-null  datetime64[ns]
 2   title     22268 non-null  object        
 3   authors   22268 non-null  object        
 4   keywords  22268 non-null  object        
 5   abstract  20601 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.0+ MB


In [5]:
year = 2020
pkl_name = os.path.join(DATA_PATH, str(year) + "_all.pkl")
df_2020 = pd.read_pickle(pkl_name)
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22994 entries, 0 to 22993
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        22994 non-null  int64         
 1   date      22994 non-null  datetime64[ns]
 2   title     22994 non-null  object        
 3   authors   22994 non-null  object        
 4   keywords  22994 non-null  object        
 5   abstract  14515 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.1+ MB


In [6]:
year = 2019
pkl_name = os.path.join(DATA_PATH, str(year) + "_all.pkl")
df_2019 = pd.read_pickle(pkl_name)
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21347 entries, 0 to 21346
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        21347 non-null  int64         
 1   date      21347 non-null  datetime64[ns]
 2   title     21347 non-null  object        
 3   authors   21347 non-null  object        
 4   keywords  21347 non-null  object        
 5   abstract  11868 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1000.8+ KB


In [7]:
# df = pd.concat([df_2023, df_2022, df_2021, df_2020, df_2019]).reset_index()
# df = df.drop(columns="index")

## Try small data set

In [19]:
df = df_2019.loc[:10000, :]

In [20]:
# df['authors'] = df['authors'].apply(lambda x: ast.literal_eval(x))
relationships = []
for idx in range(df.shape[0]):
# Add a relationship to the list for each pair of unique characters within the window
    if len(df.loc[idx, 'authors']) > 1:
        for id, a in enumerate(df.loc[idx, 'authors'][:-1]):
            b = df.loc[idx, 'authors'][id + 1]
            relationships.append({"source": a, "target": b})

In [21]:
relationship_df = pd.DataFrame(relationships)
# Sort the cases with a->b and b->a
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)

# Add a new column with value 1 to the DataFrame
relationship_df["value"] = 1

# Group the DataFrame by source and target, and sum the values
relationship_df = relationship_df.groupby(["source", "target"], sort=False, as_index=False)["value"].sum()

## Create a graph

In [22]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
# Create a graph from a pandas dataframe
G = nx.from_pandas_edgelist(relationship_df, 
                            source = "source", 
                            target = "target", 
                            edge_attr = "value", 
                            create_using = nx.Graph())

In [ ]:
pos = nx.spring_layout(G)

# Extract the edge weights from the edge_attr dictionary
edge_weights = [d["value"] for _, _, d in G.edges(data=True)]

# Scale the edge weights to the desired range and store them in a list
scaled_edge_weights = [0.1 + 5 * (w - min(edge_weights)) / (max(edge_weights) - min(edge_weights)) 
                       for w in edge_weights]

# Draw the nodes and edges with adjusted edge widths
nx.draw_networkx_nodes(G, pos, node_size=1)
nx.draw_networkx_edges(G, pos, alpha=0.5, width=scaled_edge_weights)

plt.figure(figsize=(100,100))
plt.axis('off')  # Disable axis
plt.savefig('2019.pdf',dpi=300)
plt.show()  # Show the plot


In [ ]:
def is_str_lst(lst):
  return isinstance(lst, list) and all(isinstance(elem, str) for elem in lst)
  
print(df['title'][df['title'].notnull()].apply(lambda x: isinstance(x, str)).mean())
print(df['authors'][df['authors'].notnull()].apply(is_str_lst).mean())
print(df['keywords'][df['keywords'].notnull()].apply(is_str_lst).mean())